In [7]:
from IPython.display import HTML
from data import documents, distributions
from src.vectorstore.get import retriever_ft_500
from src.agents.rag_agent import RagAgent
from langchain_openai import ChatOpenAI
import pickle
with open('testset_ragas.pkl', 'rb') as file:
    testset = pickle.load(file)
test_df = testset.to_pandas()


In [8]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

answers = []
contexts = []
llm = ChatOpenAI(model="gpt-4o", temperature=0)
agent = RagAgent(llm, retriever_ft_500)
chain = agent.get_chain()
for question in test_questions:
  response = chain.invoke({"question" : question })
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [9]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [10]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [11]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 100/100 [00:58<00:00,  1.72it/s]


In [12]:
import pandas as pd
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'other_chunking'])
df_baseline

,Metric,other_chunking
0,faithfulness,0.818649
1,answer_relevancy,0.855628
2,context_recall,0.906190
3,context_precision,0.887500
4,answer_correctness,0.743380
